# tensorflowの流れ
    1.データを用意する
    2.データをtensorflowが扱える形に変換する&正規化を行う
    3.データをトレーニング・テスト・検証用に分割する
    4.モデルのパラメータ、ハイパーパラメータを設定
    5.変数(Variable)、プレースホルダ(計算グラフへ渡すデータの位置を保持)を初期化
    6.計算グラフを作成することでモデルを定義
    7.損失関数を設定
    8.モデルの初期化・トレーニング
    9.モデルを評価
    10. 評価結果を見てハイパーパラメータを改めて設定する
    11. 未知データへの対応可能性確認
### 今回はsoftmax回帰を用いた分類を行う

    参考. https://www.tensorflow.org/versions/r1.0/get_started/mnist/beginners

In [3]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import os

In [4]:
#変数作成
my_var = tf.Variable(tf.zeros([2,3]))
sess = tf.Session()
initialize_op = tf.global_variables_initializer()
sess.run(initialize_op)

AttributeError: 'module' object has no attribute 'global_variables_initializer'

In [24]:
#プレースホルダ作成
sess = tf.Session()
#x = tf.placeholder(tf.float32, shape=[2,2])
x = tf.placeholder("float32", [2,2])
y = tf.identity(x)
x_vals = np.random.rand(2,2)
sess.run(y, feed_dict={x: x_vals})

array([[ 0.52702433,  0.99081177],
       [ 0.35060424,  0.27304938]], dtype=float32)

In [25]:
#変数初期化
#手法1
initializer_op = tf.global_variables_initializer()

#手法2
sess = tf.Session()
first_var = tf.Variable(tf.zeros([4,5]))
sess.run(first_var.initializer)
second_var = tf.Variable(tf.zeros_like(first_var))

sess.run(second_var.initializer)

AttributeError: 'module' object has no attribute 'global_variables_initializer'

In [26]:
#行列操作
id_matrix = tf.diag([1.0,1.0,1.0])
print("id_matrix \n", sess.run(id_matrix))

A = tf.truncated_normal([3,3]) #正規分布
B = tf.fill([3,4], 7) #指定した値のみの行列
C = tf.random_uniform([3,3]) #一様分布
D = tf.convert_to_tensor(np.array([[1,2,3],[4,5,6],[7,8,9]])) #入力した行列をテンソルに変換

print("A \n", sess.run(A)) #ランダム値はsess.runする時に値が決まる
print("B \n", sess.run(B))
print("C \n", sess.run(C))
print("D \n", sess.run(D))

#加減
print("A+C =\n", sess.run(A+C))
print("C-A =\n",sess.run(C-A))

#転置
print("D^t", sess.run(tf.transpose(D)))

#行列式(要素が整数ではダメっぽい)
print("Cの行列式 \n", sess.run(tf.matrix_determinant(C)))

#逆行列(中身が少数である必要有)
print("C inv \n",sess.run(tf.matrix_inverse(C)))

#コレスキー分解(正定値対称行列である必要有)
print("id_matrix コレスキー \n",sess.run(tf.cholesky(id_matrix))) #たぶん コレスキー分解, id_matrix = L L^t の Lが求まる

#固有値、固有ベクトル
#evalue, evector = sess.run(tf.self_adjoint_eig(C))
#print("\n Cの固有値 \n", evalue)
#print("\n Cの固有ベクトル\n", evector)

id_matrix 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
A 
 [[ 0.50013047 -0.43715045  1.03462803]
 [ 1.05505586 -0.15213114  0.42632279]
 [ 0.92554921  0.76519942  0.96730989]]
B 
 [[7 7 7 7]
 [7 7 7 7]
 [7 7 7 7]]
C 
 [[ 0.49207175  0.08954132  0.77718699]
 [ 0.51937878  0.07848096  0.3202256 ]
 [ 0.83184958  0.48557889  0.23328674]]
D 
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
A+C =
 [[ 1.09103346 -1.13590562  0.53409755]
 [-0.94630671  0.99397731  0.16922611]
 [-0.09946275 -0.57662952  0.95547724]]
C-A =
 [[ 0.18466556 -0.05239904  1.82512426]
 [ 0.85957718  0.31310731  0.59234285]
 [ 0.35404271  0.29028708  1.72123659]]
D^t [[1 4 7]
 [2 5 8]
 [3 6 9]]
Cの行列式 
 0.00717992
C inv 
 [[  0.92613322 -18.00481415  13.72391033]
 [ -0.34526515   4.4872117   -2.17374921]
 [  2.70120168  -0.31350413  -1.7680006 ]]
id_matrix コレスキー 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


## 1. データセットを作成(インポート)

In [27]:
from sklearn.datasets import load_digits
digits = load_digits(n_class=10)
print("elements in digits \n", dir(digits))

data = digits.data
print("\n digits.data \n", data.shape)

images = digits.images
print("\ndigits.images \n", images.shape)
target = digits.target
print("\ndigits.target(label) \n", target.shape)
target_names = digits.target_names
print("\ndigits.target_names(kind of label) \n", target_names.shape)


#今回はこっちを使う
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


elements in digits 
 ['DESCR', 'data', 'images', 'target', 'target_names']

 digits.data 
 (1797, 64)

digits.images 
 (1797, 8, 8)

digits.target(label) 
 (1797,)

digits.target_names(kind of label) 
 (10,)
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 1.1
### データセット
### 28×28=784の画像サイズのデータ 55000枚

### ラベル
### ex. 正解が3なら [0,0,0,1,0,0,0,0,0,0]

In [28]:
mnist.train.labels.shape

(55000, 10)

In [29]:
mnist.train.images.shape

(55000, 784)

In [30]:
type(mnist.train.images)

numpy.ndarray

In [32]:
#画像データを入れる箱
#x = tf.placeholder(tf.float32, [None, 784])
x = tf.placeholder("float32", [None, 784])

In [33]:
#重みとバイアスを設定
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

#0〜1に値を正規化
y = tf.nn.softmax(tf.matmul(x,W) + b) #出力値

#ネットワークで最小化すべき誤差関数としてクロスエントロピーを定義(正解データを用いる)
y_ = tf.placeholder(tf.float32, [None, 10]) #正解データ
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


In [34]:
#学習方法定義　0.5は学習率
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [35]:
sess = tf.InteractiveSession()

In [36]:
tf.initialize_all_variables().run()

In [37]:
#学習
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 3. モデル評価

In [38]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [39]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
accuracy

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

In [41]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9175


# 4. 可視化

In [ ]:
a = tf.equal([1,1,2],[1,1,3])

In [ ]:
dir(accuracy)

In [ ]:
a.value_index

In [32]:
a.consumers

<bound method Tensor.consumers of <tf.Tensor 'Equal_4:0' shape=(3,) dtype=bool>>

In [33]:
sess.run(a)

array([ True,  True, False], dtype=bool)